## Dimensionality Reduction


In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from collections import defaultdict
from sklearn import tree
from IPython.display import Image
import pydotplus
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import tree
import pydotplus
from IPython.display import Image

In [ ]:
X_train = pd.read_csv("X_train.txt", header=None,  delim_whitespace=True)
y_train = pd.read_csv("y_train.txt", header=None, delim_whitespace=True )
X_test = pd.read_csv("X_test.txt", header=None, delim_whitespace=True )
y_test = pd.read_csv("y_test.txt", header=None, delim_whitespace=True )
features = pd.read_csv("features.txt", header=None, delim_whitespace=True )
subject = pd.read_csv("subject_train.txt", header=None, delim_whitespace=True )

In [ ]:
feature = features
subjects = subject
feature.drop(0,inplace=True, axis=1)

In [ ]:
#inseriamo l'intestazione al dataset
lista=[]
feat_transpa = feature.transpose()
for i in range(561):
    lista.append(feat_transpa.iloc[0][i])
X_test.columns=lista    
X_train.columns=lista

In [ ]:
# eliminiamo le colonne che contengono la stima mad in quanto quasi uguale a dev.std
stringa="mad()"
for col in X_train.columns:
    if(stringa in col):
        X_train.drop(labels=col, axis=1, inplace=True)
for col in X_test.columns:
    if(stringa in col):
        X_test.drop(labels=col, axis=1, inplace=True)        

## Variance Threshold

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
#storing the variance and name of variables
variance = X_train.var()
columns = X_train.columns
variance

In [ ]:
index=np.arange(0,len(columns))
df=pd.DataFrame({'Columns': index, 'Variance': variance})
df1= df.sort_values(by='Variance', ascending=False)

In [ ]:
#saving the names of variables having variance more than a threshold value
variable = [ ]
for i in range(0, len(variance)):
    if variance[i]>=0.29: #setting the threshold as 1%
        variable.append(columns[i])
len(variable)        

In [ ]:
#Plot Variance vs Columns
sns.set(rc = {'figure.figsize':(7,5)})
p=sns.lineplot(data=df1, x=index,  y="Variance")
p.set_xlabel("N° cols", fontsize = 15)
p.set_ylabel("Variance", fontsize = 15)
plt.show()

In [ ]:
sel = VarianceThreshold(threshold=(0.29)) # Valore ideale come trad-off tra accuracy e n°colonne
X_train_sel= sel.fit_transform(X_train)
X_train_sel.shape #vediamo che sputa fuori 97 colonne
ciao = pd.DataFrame(X_train_sel)
#len(X_train_sel[0][:])

In [ ]:
X_test_sel = sel.transform(X_test)

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_sel, y_train)

y_pred = clf.predict(X_test_sel)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

###  Accuracy vs Var_Thresh


In [ ]:
ind=np.arange(0.01, 0.4, 0.01)
acc=[]

for i in ind:
    sel = VarianceThreshold(threshold=(i))
    X_train_sel = sel.fit_transform(X_train)
    X_test_sel = sel.transform(X_test)
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train_sel, y_train)
    y_pred = clf.predict(X_test_sel)
    acc.append(accuracy_score(y_test, y_pred))
    
#plot
sns.set(rc = {'figure.figsize':(11,7)})
p=sns.lineplot( x=ind,  y=acc)
p.set_xlabel("Var_Threshold", fontsize = 15)
p.set_ylabel("Accuracy", fontsize = 15)
plt.show()

## Univariate Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif 

In [ ]:
sel = SelectKBest(score_func=f_classif, k=30) # scegliamo il numero di feature che desideriamo secondo lo score f_classif di ANOVA
X_train_sel = sel.fit_transform(X_train, np.ravel(y_train))

X_train_sel.shape

In [ ]:
X_test_sel = sel.transform(X_test)

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_sel, y_train)

y_pred = clf.predict(X_test_sel)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

### Uivariate FS with ANOVA vs N°cols

In [ ]:
ind1=np.arange(10,100,1)
acc1=[]
for i in ind1:
    sel = SelectKBest(score_func=f_classif, k=i) 
    X_train_sel = sel.fit_transform(X_train, np.ravel(y_train))
    X_test_sel = sel.transform(X_test)
    clf1 = DecisionTreeClassifier(random_state=42)
    clf1.fit(X_train_sel, y_train)
    y_pred = clf1.predict(X_test_sel)
    acc1.append(accuracy_score(y_test, y_pred))
    
sns.set(rc = {'figure.figsize':(11,7)})
p=sns.lineplot( x=ind1,  y=acc1)
p.set_xlabel("K_features", fontsize = 15)
p.set_ylabel("Accuracy", fontsize = 15)
plt.show()

## RFE SelectFromModel

In [ ]:
from sklearn.feature_selection import SelectFromModel 

In [ ]:
sel = SelectFromModel(DecisionTreeClassifier(max_depth=None, min_samples_split=30, min_samples_leaf=30, random_state=42)) #qui selezioniamo l'algoritmo da utilizzare
X_train_sel = sel.fit_transform(X_train, y_train)
X_train_sel.shape

In [ ]:
#nome colonne salvate
features_idx = sel.get_support()
features_name = X_train.columns[features_idx]

In [ ]:
# X con risultati migliori!
X_train_new = X_train[features_name]
X_test_new = X_test[features_name] 
X_test_new

In [ ]:
X_test_sel = sel.transform(X_test)

clf = DecisionTreeClassifier(max_depth=None, min_samples_split=30, min_samples_leaf=30, random_state=42)
clf.fit(X_train_sel, y_train)

y_pred = clf.predict(X_test_sel)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

### Alternativa con randomforest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [ ]:
sel = SelectFromModel(RandomForestClassifier(), max_features=32)
X_train_sel = sel.fit_transform(X_train, np.ravel(y_train))
X_train_sel.shape

In [ ]:
sel.get_support() #To see which features are important 
select_feat= X_train.columns[(sel.get_support())] 
#select_feat

In [ ]:
X_test_sel = sel.transform(X_test)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_sel, np.ravel(y_train))

y_pred = clf.predict(X_test_sel)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

## Recursive Feature Elimination 

In [ ]:
from sklearn.feature_selection import RFE
clf_dt = DecisionTreeClassifier()
clf_dt

In [ ]:
rfe = RFE(estimator=clf, step=1)
rfe = rfe.fit(X_train, np.ravel(y_train))

In [ ]:
selected_rfe_features = pd.DataFrame({'Feature': list(X_train.columns),
                                      'Ranking': rfe.ranking_})
selected_rfe_features.sort_values(by='Ranking')

In [ ]:
a=np.where(selected_rfe_features.iloc[:,1]==1) # salvo in a le feature più importanti
np.count_nonzero(a) #conto quante sono

In [ ]:
X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)

In [ ]:
clf_model = clf.fit(X_train_rfe, np.ravel(y_train))

In [ ]:
y_pred = clf_model.predict(X_test_rfe)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

###  RFE with cross validation

In [ ]:
rfecv = RFECV(estimator=clf_dt, step=1, cv=5, scoring='accuracy')
rfecv = rfecv.fit(X_train, y_train)
print('optimal number of features:', rfecv.n_features_)
print('Best features:', X_train.columns[rfecv.support_])

In [ ]:
var = X_train.columns[rfecv.support_]
X_train_new = X_train[var]
X_test_new = X_test[var]

In [ ]:
dt = DecisionTreeClassifier()
dt1 = dt.fit(X_train_new, y_train)
y_pred = dt1.predict(X_test_new)

In [ ]:
print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

In [ ]:
y_pred = rfecv.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

## PCA (Principal Component Analysis)


In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=26)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)

In [ ]:
num_components = len(pca.explained_variance_ratio_)
indx = np.arange(num_components)
values = pca.explained_variance_ratio_
cumulative = np.cumsum(values) # in questo vettore salviamo la varianza cumulata
cumulative

In [ ]:
#questo comando ci indica i valori in stringa del vettore! *100 serve per trasformare il valore in percetuale, mentre il secondo argomento indica quantecifre decimale ci vogliono
r"%s" % ((str(values[4] *100 ) [:3])) 

### Variance explained ratio for each Principal Component

In [ ]:
#PLOT Variance Explained Ratio for each prinicpal components

plt.figure(figsize=(16,6))
ax=plt.subplot(111) #pos sarebbe il primo parametro ed è composto da 3 digit, il primo per indicare il numero di righe, il secondo le colonne e il terzo l'indice del subplot


ax.bar(indx, values)
ax.plot(indx, cumulative)

#add annotation of percentage of each component to graph 
for val in range(num_components):
    ax.annotate(r"%s" % ((str(values[val] * 100) [:3])), (indx[val], values[val]),
               va= "bottom", ha= "center", fontsize= 12)
    
#setting tick parameters    
ax.xaxis.set_tick_params(width=1)
ax.yaxis.set_tick_params(width=3, length=15)
ax.set_xlabel("Principal Component Values", size=13)
ax.set_ylabel("Variance Explained Percentage", size=13)
plt.title("Variance Explained Ratio for each prinicpal components", size=13)
plt.show()

In [ ]:
X_test_pca = pca.transform(X_test)

clf = DecisionTreeClassifier(min_samples_leaf=3, random_state=42)
clf.fit(X_train_pca, y_train)

y_pred = clf.predict(X_test_pca)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

### Cumulative_explained_variance / PC

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components',fontsize = 15, fontweight = 'bold')
plt.ylabel('cumulative explained variance',fontsize = 15, fontweight = 'bold')
#plt.xscale("symlog")
#plt.xlim(0,200,10)
plt.show()

In [ ]:
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap=plt.cm.prism, edgecolor='k', alpha=0.7)
plt.show()

## Visualizzazione PCA 2D

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)

In [ ]:
pca_df = pd.DataFrame(data = X_train_pca, columns = ['pca_1', 'pca_2'])
pca_df['label'] = y_train

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

target = [1,2,3,4,5,6]
class_name = ["WALKING","WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LAYING"]
colors=['yellow', 'black', 'cyan', 'green', 'blue', 'red']

for targets, color in zip(target,colors):
    indicesToKeep = pca_df['label'] == targets
    ax.scatter(pca_df.loc[indicesToKeep, 'pca_1']
               , pca_df.loc[indicesToKeep, 'pca_2']
               , c = color
               , s = 50)
#print((indicesToKeep))
ax.legend(class_name)
ax.grid()

## t-SNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2)
X_train_tsne = tsne.fit_transform(X_train)
X_train_tsne.shape

In [ ]:
plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], c=y_train, cmap=plt.cm.prism, edgecolor='k', alpha=0.7)
plt.show()

In [ ]:
#X_test_mds = mds.transform(X_test) # no transform

clf = DecisionTreeClassifier(min_samples_leaf=3, random_state=42)
clf.fit(X_train_tsne, y_train)

y_pred = clf.predict(X_train_tsne)

print('Accuracy %s' % accuracy_score(y_train, y_pred))
print('F1-score %s' % f1_score(y_train, y_pred, average=None))
print(classification_report(y_train, y_pred))

### Plot al variare di Perplexity e N_Iteraz

In [ ]:
# perform t-sne with different preplexities and their plots
def perform_tsne(X_data, y_data, perplexities, n_iter=300, img_name_prefix='t-sne'):
        
    for index,perplexity in enumerate(perplexities):
        # perform t-sne
        print('\nperforming tsne with perplexity {} and with {} iterations at max'.format(perplexity, n_iter))
        X_reduced = TSNE(perplexity=perplexity).fit_transform(X_data) #verbose=2
        print('Done..')
        
       # prepare the data for seaborn         
        print('Creating plot for this t-sne visualization..')
        df = pd.DataFrame({'x':X_reduced[:,0], 'y':X_reduced[:,1], 'label':y_data }) #,'label':y_data
       
       ## draw the plot in appropriate place in the grid
        sns.lmplot(data=df, x='x', y='y', hue='label',  fit_reg=False, height=7,\
                  palette="Set1") #markers=['^','v','s','o', '1','2']
        plt.title("t-Sne with perplexity : {} and max_iter : {}".format(perplexity, n_iter))
        img_name = img_name_prefix + '_perp_{}_iter_{}.png'.format(perplexity, n_iter)
        print('saving this plot as image in present working directory...')
        plt.savefig(img_name)
        plt.show()
        print('Done')

In [ ]:
classe = { 1 : "WALKING" , 2 : "WALKING_UPSTAIRS", 3 : "WALKING_DOWNSTAIRS", 4 : "SITTING", 5 : "STANDING", 6 : "LAYING"}
y_train_activity = y_train[0].map(classe)

In [ ]:
perform_tsne(X_data = X_train ,y_data=y_train_activity, perplexities =[20,50,100])